In [6]:
from create_report import *

In [7]:
file_paths = ['../../../data/2025-04-17_15-53-24-server_data.pickle', '../../../data/2025-04-17_15-58-57-server_data.pickle']
file_paths = ['../../../data/2025-04-18_18-38-28-server_data.pickle'] # checkpoint 1
file_paths = ['../../../data/2025-04-19_17-24-08-server_data.pickle'] # checkpoint 2
file_paths = ['../../../data/2025-04-19_18-54-00-server_data.pickle'] # checkpoint 2
file_paths = ['../../../data/2025-04-20_18-54-23-server_data.pickle'] # checkpoint 3
file_paths = ['../../../data/2025-04-21_15-00-46-server_data.pickle'] # checkpoint 4
file_paths = ['../../../data/2025-04-21_15-00-46-server_data.pickle', '../../../data/2025-04-21_18-00-01-server_data.pickle'] # checkpoint 4
file_paths = ['../../../data/2025-04-21_15-00-46-server_data.pickle'] # checkpoint 4
file_paths = ['../../../data/2025-04-22_19-07-16-server_data.pickle'] # exp1 no spoofing
# file_paths = ['../../../data/2025-04-23_07-48-43-server_data.pickle'] # exp1 spoofing
file_paths = ['../../../data/2025-04-18_18-38-28-server_data.pickle'] # checkpoint 1
users = {}
timestamps = []
mid_prices = []
for file_path in file_paths:
    print(f"Processing file: {file_path}")
    data = pickle_load(file_path)
    for _, order_books in data.items():
        extract_user_data(order_books, users, timestamps, mid_prices)

Processing file: ../../../data/2025-04-18_18-38-28-server_data.pickle


In [8]:
stats_df, stock_income = compute_statistics(users, mid_prices)
# print(stats_df.to_string(index=False, justify='rigth', float_format='%.2f'))
stats_df2 = stats_df.round(2)
stats_df2 = stats_df2.drop(columns=['Name'])
# censor the user
stats_df2['User'] = stats_df2['User'].apply(lambda x: f"{x[:4]}...{x[-4:]}")
# sort by FinalBalance column and reset the index
stats_df2 = stats_df2.sort_values(by='FinalBalance', ascending=False)
stats_df2 = stats_df2.reset_index(drop=True)

display(stats_df2)
print(f"  Stock fee income: {stock_income}")
# save the stats_df to a html file
stats_df2.to_html("statistics.html", index=False, justify='right', float_format='%.2f')

,User,FinalBalance,Return (%),AvgVolumePerStep,MaxVolume,AvgBalance,NumOrders
0,8c46...b0c9,11533.42,15.33,66.97,112,-5994.56,50
1,03b9...b2d2,10840.19,8.40,23.00,106,-1757.01,21
2,7b59...8582,10617.28,6.17,65.14,95,-6294.04,38
3,5a17...690b,10494.13,4.94,5.34,104,-94.38,165
4,31bd...11b6,10372.20,3.72,59.44,70,-5801.62,3
5,475e...4e14,10324.89,3.25,8.87,100,-656.98,34
6,608e...f3e6,10292.99,2.93,99.67,100,-9966.53,1
7,0d32...2ad8,10160.53,1.61,30.77,95,-3026.52,9
8,7ad7...26cb,10138.60,1.39,29.70,30,-2919.61,5
9,5f66...5c0b,10107.08,1.07,12.49,18,-1202.42,48


  Stock fee income: 6117.246699000042


In [9]:
import plotly_resampler
from plotly_resampler import FigureResampler
import plotly.graph_objects as go
import pandas as pd

def plot_best_traders_interactive(users, timestamps, mid_prices, top_10):
    # Convert nanoseconds to datetime
    try:
        timestamps = pd.to_datetime(timestamps, unit='ns')
    except Exception as e:
        print(f"Error converting timestamps: {e}")
        return

    fig = FigureResampler(go.Figure(), default_downsampler=plotly_resampler.MinMaxLTTB(parallel=True))
    
   
    user_balances = {}
    for user, data in users.items():
        if user in top_10["User"].values and len(data["balance"]) > 0:
            volume = np.array(data["volume"])
            balance = np.array(data["balance"]) + volume * mid_prices[-1]
            # Pad the balance with 0 in the beginning to match the length of timestamps
            padding = np.zeros(len(timestamps) - len(balance))  # Create padding as a numpy array
            balance = np.concatenate((padding, balance))  # Concatenate padding and balance
            user_balances[user] = balance
      
    # Add user balances to the plot
    for user in top_10["User"].values:
        user_name = f"{user[:4]}...{user[-4:]}"
        if user in user_balances:
            fig.add_trace(go.Scatter(
                x=timestamps,
                y=user_balances[user],
                mode='lines',
                name=user_name,
                # name=users[user]["name"],
                yaxis="y1"
            ))
            


    # Add mid prices to the plot
    fig.add_trace(go.Scatter(
        x=timestamps,
        y=mid_prices,
        mode='lines',
        name="Mid Price",
        line=dict(color='black'),
        opacity=0.5,
        yaxis="y2"
    ))

    # Update layout for dual y-axes
    fig.update_layout(
            title=dict(
            text="Balance of Best Traders Over Time",
            y=1,  # Move the title higher
            pad =dict(t=5),  # Add padding to the title
                
        ),
        xaxis=dict(title="Timestamp"),
        yaxis=dict(title="Balance", side="left"),
        yaxis2=dict(title="Mid Price", overlaying="y", side="right"),
        legend=dict(orientation="h", x=0.5, y=1.4, xanchor="center"),
        margin=dict(l=50, r=50, t=170, b=50),
        height=500, width=800,
    )
    
    for trace in fig.data:
        trace.name = trace.name.split("~")[0].strip()
        trace.name = trace.name.replace("[R]", "").strip()
        
    
    # Update layout for font size
    fig.update_layout(
        title=dict(
            text="Balance of Best Traders Over Time",
            y=1,
            font=dict(size=20)  # Increase title font size
        ),
        xaxis=dict(
            title="Timestamp",
            titlefont=dict(size=16),  # Increase x-axis title font size
            tickfont=dict(size=14)   # Increase x-axis tick font size
        ),
        yaxis=dict(
            title="Balance",
            titlefont=dict(size=16),  # Increase y-axis title font size
            tickfont=dict(size=14)   # Increase y-axis tick font size
        ),
        yaxis2=dict(
            title="Mid Price",
            titlefont=dict(size=16),  # Increase secondary y-axis title font size
            tickfont=dict(size=14)   # Increase secondary y-axis tick font size
        ),
        legend=dict(
            font=dict(size=14)  # Increase legend font size
        ),
        margin=dict(l=50, r=50, t=170, b=50),
        height=500, width=800,
    )

    # Show the interactive plot
    fig.show()
    # Save the plot as a pdf
    fig.write_image("best_traders_plot.pdf", format="pdf", engine="kaleido")
    # Save the interactive plot as an HTML file
    fig.write_html("best_traders_plot.html")
    # Save the interactive plot as an png file
    fig.write_image("best_traders_plot.png", format="png", engine="kaleido")

In [10]:
top_10 = stats_df.head(10)
plot_best_traders_interactive(users, timestamps, mid_prices, top_10)
